<center> <a href="https://githubtocolab.com/felixp8/lfads-nlb-tutorials/blob/main/lfads_for_nlb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> </center>

# Evaluating LFADS with the Neural Latents Benchmark '21

Neural Latents Benchmark '21 (NLB'21) is a benchmark suite aimed at standardizing evaluation of latent variable models of neural spiking activity spanning a variety of tasks and brain areas. The primary objective of the challenge is to infer the firing rates of a set of held-out neurons given the spiking activity of held-in neurons, a procedure called co-smoothing.

The benchmark suite features several datasets from experiments spanning a range of behaviors and brain regions, but they are all provided in the standard Neurodata Without Borders format and available on [DANDI](https://dandiarchive.org). The benchmark challenge itself is hosted on the platform [EvalAI](https://eval.ai), where model predictions can be submitted and automatically evaluated on private evaluation data.

To facilitate participation in the competition, we provide the code package [`nlb_tools`](https://github.com/neurallatents/nlb_tools), which has functions for data preprocessing and submission preparation.

This tutorial will walk through how to use NLB'21 to evaluate LFADS. Specifically, it will walk through:
1. **Dataset download** - getting dataset files from DANDI on to your machine
2. **Data loading and preprocessing** - using `nlb_tools` to extract the data we expect you to model
3. **Modeling neural data** - using LFADS to perform inference on the extracted data
4. **Submitting and evaluating model predictions** - packaging predictions for submission to EvalAI or local evaluation

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main.png?raw=true" width="480" /> </center>

## Part 0: Setup

First, we need to install and import the packages we need for this notebook. Note that, due to a numpy dependency mismatch, Google Colab will always ask you to restart runtime. The notebook should work fine without restarting the runtime, so you can ignore that message.

In [2]:
#@title Import Dependencies

# check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os

# set up environment
if IN_COLAB:
    ! git clone https://github.com/felixp8/lfads-nlb-tutorials.git
    os.chdir('lfads-nlb-tutorials')

# nlb_tools
if IN_COLAB:
    !pip install git+https://github.com/neurallatents/nlb_tools.git 
else:
    try:
        from nlb_tools.nwb_interface import NWBDataset
    except:
        raise ImportError('Module `nlb_tools` not found')

# DANDI CLI tool (optional, can use website instead)
if IN_COLAB:
    !pip install dandi==0.36.0
else:
    try:
        import dandi
    except:
        raise ImportError('Module `dandi` not found. ' + 
            'However, it is optional for this tutorial, as you can download the data from the DANDI website directly'
        )
  
if IN_COLAB:
    # install necessary packages 
    ! git clone https://github.com/snel-repo/autolfads-tf2.git
    os.chdir('autolfads-tf2')
    ! pip install -e lfads-tf2
    ! pip install -e tune-tf2
    os.chdir('..')
    # add lfads_tf2
    os.environ['PYTHONPATH'] += ":/content/autolfads-tf2"
else:
    try:
        from lfads_tf2.utils import restrict_gpu_usage
    except:
        x = input('autolfads-tf2 is not installed. ' + 
            'We recommend setting up a conda environment and installing the package. ' + 
            'However, it can also be installed from the notebook. ' + 
            'Would you like to install it now? (y/n)'
        )
        if (x.strip() in ['Y', 'y']):
            ! git clone https://github.com/snel-repo/autolfads-tf2.git
            os.chdir('autolfads-tf2')
            ! pip install -e .
            os.chdir('..')
            from lfads_tf2.utils import restrict_gpu_usage
        else:
            raise ImportError('Module `lfads_tf2` not found')

import numpy as np
import h5py
import warnings
warnings.simplefilter (action='ignore', category=FutureWarning)

## Part 1: Data Preparation

### 1.1. Dataset Download

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_download.png?raw=true" width="800" /> </center>

The datasets are available on the platform DANDI. They can be downloaded directly from the website or by using the DANDI CLI tool, as shown below. For this notebook, we will be using the MC_Maze_Large dataset, which is available from [here](https://dandiarchive.org/dandiset/000138). Links to the other datasets can be found on [our website](https://neurallatents.github.io/datasets).

In [4]:
!dandi download https://dandiarchive.org/dandiset/000138

The above line will download two files into the folder `./000138/sub-Jenkins/`. Next, we'll get the path of the downloaded files and list them.

In [5]:
import os
curr_path = os.getcwd()
fpath = curr_path + '/000138/sub-Jenkins/'
os.listdir(fpath) 

['sub-Jenkins_ses-large_desc-test_ecephys.nwb',
 'sub-Jenkins_ses-large_desc-train_behavior+ecephys.nwb']

The file with 'desc-train' in its name is for training, while the file with 'desc-test' in its name is for final model evaluation. As we take a look at the data, we will see the differences between these two files.

### 1.2. Dataset Loading

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_dataload.png?raw=true" width="800" /> </center>

To get the NWB data into Python, we provide the `NWBDataset` class, which can load from the dataset files and perform simple preprocessing operations. To load a dataset, you instantiate an instance of NWBDataset and provide the path to the files.

In [7]:
from nlb_tools.nwb_interface import NWBDataset
dataset = NWBDataset(fpath=fpath) 

### 1.3. Data Format

The loaded data are primarily stored in two pandas DataFrames: `NWBDataset.data` and `NWBDataset.trial_info`.

#### `NWBDataset.data`
`NWBDataset.data` contains the continuous recorded data, like spike counts and kinematics. Each row consists of measurements taken at a particular timestep. Most importantly, spiking data from held-in units are labeled `spikes` and data from held-out units is labeled `heldout_spikes`.

In the training data, all fields are available:

In [8]:
dataset.data.iloc[100000:100100]

signal_type            cursor_pos           eye_pos        hand_pos  \
channel                         x         y       x     y         x   
clock_time                                                            
0 days 00:01:40         -5.200000  3.300000     0.6   1.1 -5.195095   
0 days 00:01:40.001000  -5.199120  3.299442     2.5   0.6 -5.196711   
0 days 00:01:40.002000  -5.198598  3.299110     2.5   0.4 -5.198551   
0 days 00:01:40.003000  -5.198598  3.299110     2.7   0.5 -5.200457   
0 days 00:01:40.004000  -5.199120  3.299442     2.8   0.8 -5.202659   
...                           ...       ...     ...   ...       ...   
0 days 00:01:40.095000  -5.200000  3.300000     1.6   8.8 -5.239548   
0 days 00:01:40.096000  -5.193734  3.308271     1.7   9.1 -5.240553   
0 days 00:01:40.097000  -5.188833  3.311073     0.0  10.0 -5.241476   
0 days 00:01:40.098000  -5.187298  3.309505     0.3   9.7 -5.242439   
0 days 00:01:40.099000  -5.190729  3.305204     0.2   9.7 -5.243523   

signal_type                        hand_vel           heldout_spikes       \
channel                         y         x         y           1031 1051   
clock_time                                                                  
0 days 00:01:40        -31.606258 -1.481366  0.261386            0.0  0.0   
0 days 00:01:40.001000 -31.605926 -1.727835  0.317635            0.0  0.0   
0 days 00:01:40.002000 -31.605623 -1.873343  0.342863            0.0  0.0   
0 days 00:01:40.003000 -31.605240 -2.053902  0.334682            0.0  0.0   
0 days 00:01:40.004000 -31.604953 -2.238392  0.280908            0.0  0.0   
...                           ...       ...       ...            ...  ...   
0 days 00:01:40.095000 -31.618091 -1.010222 -1.636778            0.0  0.0   
0 days 00:01:40.096000 -31.619778 -0.964223 -1.878082            0.0  0.0   
0 days 00:01:40.097000 -31.621848 -0.942786 -2.159794            0.0  0.0   
0 days 00:01:40.098000 -31.624097 -1.023439 -2.277696            0.0  0.0   
0 days 00:01:40.099000 -31.626403 -0.917548 -2.321146            0.0  0.0   

signal_type             ... spikes                                          \
channel                 ...   2741 2743 2761 2771 2781 2791 2801 2881 2941   
clock_time              ...                                                  
0 days 00:01:40         ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.001000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.002000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.003000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.004000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                     ...    ...  ...  ...  ...  ...  ...  ...  ...  ...   
0 days 00:01:40.095000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.096000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.097000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
0 days 00:01:40.098000  ...    0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0   
0 days 00:01:40.099000  ...    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

signal_type                  
channel                2951  
clock_time                   
0 days 00:01:40         0.0  
0 days 00:01:40.001000  0.0  
0 days 00:01:40.002000  0.0  
0 days 00:01:40.003000  0.0  
0 days 00:01:40.004000  0.0  
...                     ...  
0 days 00:01:40.095000  0.0  
0 days 00:01:40.096000  0.0  
0 days 00:01:40.097000  0.0  
0 days 00:01:40.098000  0.0  
0 days 00:01:40.099000  0.0  

[100 rows x 170 columns]

In the test data, only held-in spikes are available, while other data is concealed with NaNs:

In [9]:
dataset.data.iloc[1000:1100]

signal_type            cursor_pos     eye_pos     hand_pos     hand_vel      \
channel                         x   y       x   y        x   y        x   y   
clock_time                                                                    
0 days 00:00:01               NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.001000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.002000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.003000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.004000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
...                           ...  ..     ...  ..      ...  ..      ...  ..   
0 days 00:00:01.095000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.096000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.097000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.098000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   
0 days 00:00:01.099000        NaN NaN     NaN NaN      NaN NaN      NaN NaN   

signal_type            heldout_spikes       ... spikes                      \
channel                          1031 1051  ...   2741 2743 2761 2771 2781   
clock_time                                  ...                              
0 days 00:00:01                   NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.001000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.002000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.003000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.004000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...    ...  ...  ...  ...  ...   
0 days 00:00:01.095000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.096000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.097000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.098000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   
0 days 00:00:01.099000            NaN  NaN  ...    0.0  0.0  0.0  0.0  0.0   

signal_type                                      
channel                2791 2801 2881 2941 2951  
clock_time                                       
0 days 00:00:01         0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.001000  0.0  0.0  1.0  0.0  0.0  
0 days 00:00:01.002000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.003000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.004000  0.0  0.0  0.0  0.0  0.0  
...                     ...  ...  ...  ...  ...  
0 days 00:00:01.095000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.096000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.097000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.098000  0.0  0.0  0.0  0.0  0.0  
0 days 00:00:01.099000  0.0  0.0  0.0  0.0  0.0  

[100 rows x 170 columns]


#### `NWBDataset.trial_info`
Each row of `trial_info` contains information about a particular experimental trial, such as when it begins and ends. As with the `NWBDataset.data`, almost all information is concealed in the test data. The field `split`, common to all of our provided datasets, indicates what data split a trial belongs to (explained in more detail in Section 2.4).

In [10]:
dataset.trial_info

,trial_id,start_time,end_time,move_onset_time,split,trial_type,trial_version,maze_id,success,target_on_time,go_cue_time,rt,delay,num_targets,target_pos,num_barriers,barrier_pos,active_target
0,0,0 days 00:00:00,0 days 00:00:00.700000,0 days 00:00:00.250000,test,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0 days 00:00:00.800000,0 days 00:00:01.500000,0 days 00:00:01.050000,test,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0 days 00:00:01.600000,0 days 00:00:02.300000,0 days 00:00:01.850000,test,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0 days 00:00:02.400000,0 days 00:00:03.100000,0 days 00:00:02.650000,test,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0 days 00:00:03.200000,0 days 00:00:03.900000,0 days 00:00:03.450000,test,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,0 days 00:25:49.600000,0 days 00:25:52.636000,0 days 00:25:51.501000,train,8.0,1.0,38.0,True,0 days 00:25:50.405000,0 days 00:25:51.153000,348.0,748.0,1.0,"[[-105, 76]]",9.0,"[[74, -102, 11, 53], [86, -44, 14, 11], [103, ...",0.0
596,596,0 days 00:25:52.700000,0 days 00:25:55.746000,0 days 00:25:54.595000,train,11.0,2.0,80.0,True,0 days 00:25:53.467000,0 days 00:25:54.116000,479.0,649.0,3.0,"[[123, -81], [-130, -13], [123, 71]]",8.0,"[[-65, -15, 14, 51], [-79, -55, 55, 6], [-103,...",2.0
597,597,0 days 00:25:55.800000,0 days 00:25:58.801000,0 days 00:25:57.701000,val,7.0,2.0,37.0,True,0 days 00:25:56.545000,0 days 00:25:57.410000,291.0,865.0,3.0,"[[124, -79], [103, 83], [-105, 76]]",9.0,"[[74, -102, 11, 53], [86, -44, 14, 11], [103, ...",1.0
598,598,0 days 00:25:58.900000,0 days 00:26:01.956000,0 days 00:26:00.777000,train,7.0,1.0,37.0,True,0 days 00:25:59.613000,0 days 00:26:00.479000,298.0,866.0,1.0,"[[103, 83]]",9.0,"[[74, -102, 11, 53], [86, -44, 14, 11], [103, ...",0.0


### 1.4. Data Splits

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/data_splits.png?raw=true" width="480" /> </center>

The dataset is divided into train, val, and test splits. The train and val splits are contained within the training data file, while the test split is entirely in the test data file. This means that all data fields are available in the train and val splits, but only held-in data is available in the test split. The NLB'21 challenge has two phases based on these splits:
1. In the Validation Phase, models will be evaluated on their val split predictions. This phase is offered for sanity checking results and building familiarity with the EvalAI platform.
2. In the Test Phase, models will be evaluated on their test split predictions. This is the phase that is displayed on the public leaderboard. In this phase, the val split does not strictly need to be used for model validation, despite its name.

In this notebook, we will prepare a submission for the Validation Phase, though the code can be easily modified for the Test Phase.

### 1.5. Data Extraction

#### 1.5.1. Resampling


<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_resample.png?raw=true" width="800" /> </center>

The raw data is at 1 ms resolution, but the NLB'21 challenge expects submissions to be at 5 ms resolution, so we will resample the data before doing any other processing.

In [12]:
print(f'Data shape: {dataset.data.shape}')
print(f'Bin width: {dataset.bin_width} ms')
dataset.resample(5)
print(f'Resampled data shape: {dataset.data.shape}')
print(f'Resampled bin width: {dataset.bin_width} ms')

Data shape: (1565021, 170)
Bin width: 1 ms
Resampled data shape: (313005, 170)
Resampled bin width: 5 ms


#### 1.5.2. Trial Alignment

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_alignment.png?raw=true" width="800" /> </center>

To simplify data preparation, we abstract other formatting steps into the functions `make_train_input_tensors` and `make_eval_input_tensors`. These wrapper functions format the raw data into tensors to be used for model training and evaluation. The primary processing performed by the functions is trial alignment.

Trial alignment involves choosing a particular trial event, such as a go cue, and taking a fixed window of data around each occurrence of that event. For all of the datasets in NLB'21, we have chosen trial alignments based on the experimental design and past analyses on the data. For the MC_Maze_Large dataset, the selected trial alignment is 250 ms before to 450 ms after movement onset. Our wrapper functions will apply this alignment when given the correct dataset name.

In [14]:
from nlb_tools.make_tensors import make_train_input_tensors, make_eval_input_tensors

`make_train_input_tensors` extracts the data available for model training. 

In [15]:
train_dict = make_train_input_tensors(dataset=dataset, 
                                      dataset_name='mc_maze_large', 
                                      trial_split='train', # trial_split=['train', 'val'], for Test phase
                                      save_file=False, 
                                      include_forward_pred=True)

`make_eval_input_tensors` extracts the data used to evaluate the model. 

In [16]:
eval_dict = make_eval_input_tensors(dataset=dataset,
                                    dataset_name='mc_maze_large',
                                    trial_split='val', # trial_split='test', for Test phase
                                    save_file=False)

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_naming.png?raw=true" width="600" /> </center>

Both `make_train_input_tensors` and `make_eval_input_tensors` return dictionaries of tensors. 

The training dictionary contains:
- train_spikes_heldin - spiking activity of held-in units on training trials 
- train_spikes_heldout - spiking activity of held-out units on training trials
- train_spikes_heldin_forward - spiking activity of held-in units immediately after the trial period
- train_spikes_heldout_forward - spiking activity of held-iout units immediately after the trial period

The four different sets of data are visualized in the above figure. Each set of data is a 3D array with dimensions Trial x Time x Channel. 

<!---The tensor naming conventions are fairly straightforward. The tensors labeled 'heldin' contain spiking activity from held-in units. The tensors labeled 'heldout' contain spiking activity from held-out units. The tensors labeled 'forward' contain additional spiking activity occurring after each aligned trial window. All tensors have dimensions Batch x Time x Channel. --->

In [18]:
print(train_dict.keys())

dict_keys(['train_spikes_heldin', 'train_spikes_heldout', 'train_spikes_heldin_forward', 'train_spikes_heldout_forward'])


In [19]:
print(train_dict['train_spikes_heldin'].shape)
print(train_dict['train_spikes_heldout'].shape)
print(train_dict['train_spikes_heldin_forward'].shape)
print(train_dict['train_spikes_heldout_forward'].shape)

(375, 140, 122)
(375, 140, 40)
(375, 40, 122)
(375, 40, 40)


The shapes above indicate that there are 375 training trials, 140 time bins during the trial, 40 time bins after the trial, 122 held-in units, and 40 held-out units in this dataset.

Next, we look at the data used for evaluation.

In [20]:
print(eval_dict.keys())

dict_keys(['eval_spikes_heldin', 'eval_spikes_heldout'])


In [21]:
print(eval_dict['eval_spikes_heldin'].shape)

(125, 140, 122)


The output tensors of `make_eval_input_tensors` follow the same dimension ordering and naming conventions as those in `train_dict`. The only tensor that will always be returned in `eval_dict` is `'eval_spikes_heldin'`, as that is the only data available in the test split.

If you are using a language other than Python for your model, you will want to save these tensors as HDF5 files by changing the `save_file=False` lines in the above examples to `save_file=True`. The HDF5 files will have the same key-value pairs as the dicts and can be loaded into other programs like [MATLAB](https://www.mathworks.com/help/matlab/import_export/importing-hierarchical-data-format-hdf5-files.html) or [R](https://www.bioconductor.org/packages/devel/bioc/vignettes/rhdf5/inst/doc/rhdf5.html) scripts.

## Part 2: Modeling


<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_modeling.png?raw=true" width="480" /> </center>

Now, we will apply LFADS to the challenge. 

### 2.1. Import LFADS packages

We first import the necessary dependencies for LFADS.

In [22]:
### Import LFADS dependencies and necessary modules

from lfads_tf2.utils import restrict_gpu_usage
restrict_gpu_usage(gpu_ix=0)

from lfads_tf2.models import LFADS
from lfads_tf2.utils import load_data, merge_chops, load_posterior_averages
from lfads_tf2.defaults import get_cfg_defaults
from lfads_tf2.tuples import LoadableData

import os.path as path
import shutil

import ray
import yaml
from lfads_tf2.utils import flatten
from ray import tune
from tune_tf2.models import create_trainable_class
from tune_tf2.pbt.hps import HyperParam
from tune_tf2.pbt.schedulers import MultiStrategyPBT
from tune_tf2.pbt.trial_executor import SoftPauseExecutor

### 2.2. Input formatting


<center> <img src="https://github.com/felixp8/lfads-nlb-tutorials/blob/main/images/lfads_nlb.png?raw=true" width="800" /> </center>

We will next prepare our input and target data for training and evaluating LFADS. As seen in the figure above, we want the model to take held-in activity as input and predict firing rates for not only that held-in activity, but also held-out activity and future timesteps.

Our training input data will have held-in and held-out data for in-trial and future timesteps, so that the model can learn the relationships between held-in and held-out channels and learn to forecast.

In [24]:
tlen = train_dict['train_spikes_heldin'].shape[1]
num_heldin = train_dict['train_spikes_heldin'].shape[2]
num_heldout = train_dict['train_spikes_heldout'].shape[2]
fp_steps = train_dict['train_spikes_heldin_forward'].shape[1]
train_spikes = np.hstack([
    np.dstack([train_dict['train_spikes_heldin'], train_dict['train_spikes_heldout']]),
    np.dstack([train_dict['train_spikes_heldin_forward'], train_dict['train_spikes_heldout_forward']]),
])

For the input for evaluation, we will fill the array with zeros in place of the held-out channels and future timesteps, as we do not have access to the data. LFADS will need to predict that data without having seen it.

In [25]:
num_trials = len(eval_dict['eval_spikes_heldin'])
eval_spikes = np.hstack([
    np.dstack([eval_dict['eval_spikes_heldin'], np.full((num_trials, tlen, num_heldout), 0.0)]),
    np.full((num_trials, fp_steps, num_heldin + num_heldout), 0.0),
])

Now, we need to save these files in the format that LFADS expects. As you may recall from [Running LFADS](), we will need to split the data into training and validation splits and save them in HDF5 files. 

We will have to create a "train" and "val" split for the evaluation data as well for compatibility with LFADS dataloading functions. However, because the model is not trained on these data, the splits are not meaningful and both sets of data will simply be passed through once.

In [ ]:
valid_ratio = 0.2 # 20% of trials for validation

num_trials = len(train_spikes)
valid_inds = np.arange(0, num_trials, int(round(1./valid_ratio)))
train_inds = np.delete(np.arange(num_trials), valid_inds)

with h5py.File('./data/mc_maze_large_train_lfads.h5', 'w') as h5file:
    h5file.create_dataset('train_inds', data=train_inds)
    h5file.create_dataset('valid_inds', data=valid_inds)
    h5file.create_dataset('train_data', data=train_spikes[train_inds])
    h5file.create_dataset('valid_data', data=train_spikes[valid_inds])

num_trials = len(eval_spikes)
valid_inds = np.arange(0, num_trials, int(1./valid_ratio))
train_inds = np.delete(np.arange(num_trials), valid_inds)

with h5py.File('./data/mc_maze_large_eval_lfads.h5', 'w') as h5file:
    h5file.create_dataset('train_inds', data=train_inds)
    h5file.create_dataset('valid_inds', data=valid_inds)
    h5file.create_dataset('train_data', data=eval_spikes[train_inds])
    h5file.create_dataset('valid_data', data=eval_spikes[valid_inds])

### 2.3. Model Training

As in [Running LFADS](), you can either train a single LFADS model or train multiple to optimize hyperaparameters with AutoLFADS. We demonstrate both below. The YAML config files we use can be found in the `models/config/` directory.

#### 2.3.1. LFADS single model

In [ ]:
# build LFADS model
# cfg_path = path.join('models/config/', 'lorenz.yaml')
# model = LFADS(cfg_path=cfg_path)
# train model for a bit
# model.train()

#### 2.3.2. AutoLFADS

> **Note:** AutoLFADS is computationally expensive and it is not recommended to try running it in a single-GPU Google Colab environment. 

In [ ]:
# # the default configuration file for the LFADS model
# CFG_PATH = path.join('models/config/', "lorenz.yaml")
# # the directory to save PBT runs (usually '~/ray_results')
# PBT_HOME = path.expanduser("./ray_results")
# # the name of this PBT run (run will be stored at {PBT_HOME}/{PBT_NAME})
# RUN_NAME = "lorenz_run"  # the name of the PBT run
# # the dataset to train the PBT model on
# DATA_DIR = (
#     "data/"
# )
# DATA_PREFIX = "lfads"

# # the number of workers to use - make sure machine can handle all
# NUM_WORKERS = 2
# # the resources to allocate per model
# RESOURCES_PER_TRIAL = {"cpu": 2, "gpu": 0.5}
# # the hyperparameter space to search
# HYPERPARAM_SPACE = {
#     "TRAIN.LR.INIT": HyperParam(
#         1e-5, 5e-3, explore_wt=0.3, enforce_limits=True, init=0.004
#     ),
#     "MODEL.DROPOUT_RATE": HyperParam(
#         0.0, 0.6, explore_wt=0.3, enforce_limits=True, sample_fn="uniform"
#     ),
#     "MODEL.CD_RATE": HyperParam(
#         0.01, 0.7, explore_wt=0.3, enforce_limits=True, init=0.5, sample_fn="uniform"
#     ),
#     "TRAIN.L2.GEN_SCALE": HyperParam(1e-4, 1e-0, explore_wt=0.8),
#     "TRAIN.L2.CON_SCALE": HyperParam(1e-4, 1e-0, explore_wt=0.8),
#     "TRAIN.KL.CO_WEIGHT": HyperParam(1e-6, 1e-4, explore_wt=0.8),
#     "TRAIN.KL.IC_WEIGHT": HyperParam(1e-5, 1e-3, explore_wt=0.8),
# }
# PBT_METRIC = "smth_val_nll_heldin"
# EPOCHS_PER_GENERATION = 25

# # setup the data hyperparameters
# dataset_info = {"TRAIN.DATA.DIR": DATA_DIR, "TRAIN.DATA.PREFIX": DATA_PREFIX}
# # setup initialization of search hyperparameters
# init_space = {name: tune.sample_from(hp.init) for name, hp in HYPERPARAM_SPACE.items()}
# # load the configuration as a dictionary and update for this run
# flat_cfg_dict = flatten(yaml.full_load(open(CFG_PATH)))
# flat_cfg_dict.update(dataset_info)
# flat_cfg_dict.update(init_space)

# # Set the number of epochs per generation
# tuneLFADS = create_trainable_class(EPOCHS_PER_GENERATION)
# # connect to Ray cluster or start on single machine
# ray.init(address=None)
# # create the PBT scheduler
# scheduler = MultiStrategyPBT(HYPERPARAM_SPACE, metric=PBT_METRIC)
# # Create the trial executor
# executor = SoftPauseExecutor(reuse_actors=True)
# # Create the command-line display table
# reporter = tune.CLIReporter(metric_columns=["epoch", PBT_METRIC])
# try:
#     # run the tune job, excepting errors
#     tune.run(
#         tuneLFADS,
#         name=RUN_NAME,
#         local_dir=PBT_HOME,
#         config=flat_cfg_dict,
#         resources_per_trial=RESOURCES_PER_TRIAL,
#         num_samples=NUM_WORKERS,
#         sync_to_driver="# {source} {target}",  # prevents rsync
#         scheduler=scheduler,
#         progress_reporter=reporter,
#         trial_executor=executor,
#         verbose=1,
#         reuse_actors=True,
#     )
# except tune.error.TuneError:
#     pass

# # load the results dataframe for this run
# pbt_dir = path.join(PBT_HOME, RUN_NAME)
# df = tune.Analysis(pbt_dir).dataframe()
# df = df[df.logdir.apply(lambda path: "best_model" not in path)]
# # find the best model
# best_model_logdir = df.loc[df[PBT_METRIC].idxmin()].logdir
# best_model_src = path.join(best_model_logdir, "model_dir")
# # copy the best model somewhere easy to find
# best_model_dest = path.join(pbt_dir, "best_model")
# shutil.copytree(best_model_src, best_model_dest)

#### 2.3.3. Loading a trained model

To save time, we can load a pre-trained model instead. First, we'll download it from the GitHub repo and then we'll use it to load the trained LFADS model, as demonstrated in [Running LFADS]()

In [26]:
model_dir = 'models/lfads_mc_maze_large/'
model = LFADS(model_dir=model_dir)

<All keys matched successfully>

### 2.4. Model Inference

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_inference.png?raw=true" width="480" /> </center>

Finally, we'll generate our training and evaluation predictions by passing the data through the model.

In [28]:
training_predictions, *_ = model.sample_and_average(save=False, merge_tv=True)

loadpath = f'./data/mc_maze_large_eval_lfads.h5'
h5file = h5py.File(loadpath, 'r')
test_data = LoadableData(
    train_data=h5file['train_data'][()].astype(np.float32),
    valid_data=h5file['valid_data'][()].astype(np.float32),
    train_ext_input=None,
    valid_ext_input=None,
    train_inds=h5file['train_inds'][()].astype(np.float32),
    valid_inds=h5file['valid_inds'][()].astype(np.float32),
)
h5file.close()

eval_predictions, *_ = model.sample_and_average(loadable_data=test_data, merge_tv=True, save=False)

## Part 3: Evaluation

### 3.1. Online evaluation on EvalAI

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_submission.png?raw=true" width="480" /> </center>

#### 3.1.1. File Preparation

Now that we have predictions for the training and evaluation data, we can prepare a submission. The submission has a similar format to the returned data tensor dictionaries, but with an additional layer specifying the dataset.

The dataset name and array names must be correct in order for the automated evaluation to work properly, as shown below. `'eval_rates_heldin_forward'` and `'eval_rates_heldout_forward'` are required only if you would like results on the optional forward prediction metric.

In [30]:
tlen = train_dict['train_spikes_heldin'].shape[1]
num_heldin = train_dict['train_spikes_heldin'].shape[2]

submission = {
    'mc_maze_large': {
        'train_rates_heldin': training_predictions[:, :tlen, :num_heldin],
        'train_rates_heldout': training_predictions[:, :tlen, num_heldin:],
        'eval_rates_heldin': eval_predictions[:, :tlen, :num_heldin],
        'eval_rates_heldout': eval_predictions[:, :tlen, num_heldin:],
        'eval_rates_heldin_forward': eval_predictions[:, tlen:, :num_heldin],
        'eval_rates_heldout_forward': eval_predictions[:, tlen:, num_heldin:]
    }
}

These dicts must be in an HDF5 format to be submitted to EvalAI. We have a function called `save_to_h5` to save these dictionaries to HDF5 files while preserving the structure.

In [31]:
# from nlb_tools.make_tensors import save_to_h5
 
# save_to_h5(submission, 'submission.h5')

#### 3.1.2. Submission Upload

The files can be submitted through the EvalAI website or using their CLI tool. The CLI tool is recommended for large files (>300 MB), but there is no difference for smaller files.

In [32]:
# Configure EvalAI-CLI with your account credentials
# !evalai set_token <auth_token>

# Our challenge's id is 1256, and the phase ids are 2539 for Validation and 2540 for Test
# So, to submit to the Validation phase of NLB'21:
# !evalai challenge 1256 phase 2539 submit --file submission.h5

# and if the file is large:
# !evalai challenge 1256 phase 2539 submit --file submission.h5 --large

See [this page](https://cli.eval.ai/) for more info on the EvalAI-CLI tool.

Once your file is submitted, you can log in to EvalAI, go to our [challenge](https://eval.ai/web/challenges/challenge-page/1256/overview), and view the evaluation results in the 'My Submissions' tab. If your submission errored in evaluation, you can see the error output to assist in debugging.

### 3.2. Local evaluation

<center> <img src="https://github.com/neurallatents/nlb_workshop/blob/main/nlb_technical/img/tutorial_diagram_main_evaluation.png?raw=true" width="600" /> </center>

For the Test Phase, submissions must be uploaded to EvalAI for evaluation. For the Validation Phase, submissions can also be evaluated locally with provided data and functions.

First, we prepare the data used for evaluation with `make_eval_target_tensors`. This function extracts all necessary evaluation data from the loaded dataset.

In [33]:
from nlb_tools.make_tensors import make_eval_target_tensors

target_dict = make_eval_target_tensors(dataset=dataset, 
                                       dataset_name='mc_maze_large',
                                       train_trial_split='train',
                                       eval_trial_split='val',
                                       include_psth=True,
                                       save_file=False)

Then, we can evaluate with the `evaluate` function. Every submission is scored on a number of metrics, each evaluating different aspects of the model output.

In [34]:
from nlb_tools.evaluation import evaluate
from pprint import pprint

pprint(evaluate(target_dict, submission))

[{'mc_maze_scaling_split': {'[500] co-bps': 0.3211858864626676,
   '[500] vel R2': 0.856816843986107,
   '[500] psth R2': 0.5743404432814287,
   '[500] fp-bps': 0.18072337679652054}}]

### 3.3. Evaluation metrics

We evaluate submitted rate predictions on a number of metrics, as shown above. Each metric gauges a different aspect of model performance, providing a multi-faceted perspective on where models excel.

#### 3.3.1. Co-smoothing

Our primary metric is co-smoothing, as described above. To recap, co-smoothing evaluates a model's ability to predict the activity of a held-out set of neurons from just the activity of provided neurons. The rate predictions for held-out neurons are evaluated based on the negative log-likelihood of the observed spiking activity given predicted firing rates. The final score is standardized by the average firing rate of each neuron to compute a bits per spike value.

The assumption behind co-smoothing is that the latent variables underlying held-out neuron activity can be inferred from the available neurons. It has been shown that latent representations are distributed across many neurons in a population, providing some support for this assumption.The advantage of co-smoothing is that it requires no additional data and makes very weak assumptions about the neural activity. It relies on already-available spiking activity, and it essentially only assumes that activity of neurons in a population are related. This makes it generally applicable across datasets that span experimental settings and brain areas. However, likelihood scores vary substantially across datasets due to differences in neuron count and firing rate statistics, making comparisons of performance across datasets difficult.

#### 3.3.2. Decoding accuracy

To evaluate decoding performance, we train a linear regression model from predicted firing rates to a behavioral variable, specifically arm velocity. The outputs of the regression model are evaluated with $R^2$, or variance explained, which takes a value between $(-\infty, 1]$.

Decoding measures how much information about a given external behavioral variable is contained in the rate predictions. This gives a very concrete and relevant measure of model performance, with practical applications for brain-computer interfaces. However, it is limiting in that it ignores other information that may be encoded in neural activity but which may not be as clearly related to externally measurable behavior. This is especially problematic in higher-order brain regions, like DMFC, where brain activity is believed to related to complex cognitive functions like problem solving or time interval estimation, which do not have moment-by-moment behavioral correlates.

#### 3.3.3. Match to PSTH

Traditional analyses of single-neuron firing rates relied on averaging smoothed spike counts across trials of the same condition, resulting in estimates of a neuron's true firing rate within a given condition. Our match to PSTH metric computes true PSTHs from spiking activity and compares it to average model firing rate predictions across trials of the same condition, evaluating the model predictions with $R^2$, or variance explained.

Our implementation of match to PSTH gauges how well model predictions distinguish experimental conditions. However, by averaging across trials, the metric ignores single-trial variance. Models that accurately capture single-trial variance and models that fail to could perform equally well on this metric. As a result, it does a poor job distinguishing models past a particular level of performance, and it ignores an important advantage of more powerful single-trial modeling approaches.

#### 3.3.4. Forward prediction

Forward prediction tasks the models with predicting another 200 ms of data after each trial window. The predictions are evaluated by negative log-likelihood of spiking, just like co-smoothing evaluation. This evaluation includes both held-in and held-out neurons, as no activity for these future timesteps is provided as input to the model.

Forward prediction assumes that future activity is predictable, which may not always hold true. In particular, datasets like MC_RTT, where new random inputs can occur unexpectedly at any point within and after a trial window, it should not be possible to predict neural activity after the end of the trial window. However, for datasets like MC_Maze, where dynamics during the reach are well-modeled as autonomous, future prediction is a reasonable measure for the quality of the model's inferred dynamical rules.

## Summary and Conclusions

This tutorial gave an overview of the NLB'21 pipeline from loading raw data to submission and demonstrated how to make use of our provided code package, `nlb_tools`, to participate in NLB'21 with LFADS. It also described the principles motivating our evaluation strategy, which can be applied generally beyond the scope of just the benchmark.

For additional helpful resources on NLB'21, we have a number of other tutorials and example scripts covering a variety of topics:
* The notebooks in the [`nlb_tools` repo](https://github.com/neurallatents/nlb_tools) demonstrate application of classical methods like spike smoothing, GPFA, and SLDS to NLB'21.
* Andrew Sedler's [nlb-lightning](https://github.com/arsedler9/nlb-lightning) package provides a convenient framework to develop and evaluate PyTorch Lightning models for NLB'21.